In [1]:
###To import the necessary libaries 

import pandas as pd
import numpy as np
from gurobipy import *

In [2]:
#########Parameters Set-up (in english)############

# qt = quantity of catfish produced (in pound) in each month t 
# xt = quantity of catfish (in pound) that will be kept as inventory in each month t
# min_productiont = minimum quantity of catfish (in pound) that the producer needs to commit to produce in month t 
# (except month 0 for the planning month) = 60000 pounds/month 
# dt = quantity of catfish (in pound) that will be sold to the intermediaries  
# ht = holding cost of catfish ($/pound) will be incurred if there is inventory to be kept at the end of month t = $0.036/pound
# ft = fixed cost of running the farm for each month t ($/month) = $13,260.66667/month
# var_costt = all variable cost except the feed cost ($/pound)  incurred for production in each month t 
# (except month 0 for the planning month) = $0.2051444222/pound
# e_onet = quantity of feed 1 used in each month t (pound/month) (except month 0 for the planning month) 
# e_twot = quantity of feed 2 used in each month t (pound/month) (except month 0 for the planning month)
# pe_onet = price of feed 1 ($/pound) for each month t (except month 0 for the planning month)
# pe_twot = price of feed 2 ($/pound) for each month t (except month 0 for the planning month)
# p_onet = the pessimistic price ($/pound) from game theory model = $0.70665/pound
# p_twot = the Laplace price ($/pound) from game theory model for only Feb 2022(02) and March 2022(03) 
# otherwise it will be the pessimistic price = $0.7852/pound 
# max_production = farm's max production capacity is = 96,000 pounds/month 
# max_production_budget = the budget for the farm's production in 1 year ($) = $673,719 in 1 year
# t = {0, 1, 2, …, 13}  where 0: Dec 2021 (planning stage so no production but there is an inventory capped for Jan 2022), 
#                                            1: Jan 2022, …, 13: Dec 2022

In [3]:
[0]+[0.9]*12

[0, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9, 0.9]

In [4]:
#########Parameters Set-up############

# Production, budget and others 
max_production = 96000
min_production = [0]+[60000]*12
max_production_budget = 6737190
#max_production_budget = 673719
fcr = 2.2

# Cost
h = [0.036]*13 
var_cost = [0]+ [0.2051444222]*12
f = 13260.66667

# Prices for the different feed 
#pe_one = [0, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105, 0.105]
pe_one = [0, 0.105, 0.10, 0.10, 0.2, 0.105, 0.2, 0.105, 0.2, 0.105, 0.2, 0.105, 0.10]
#pe_two = pe_one
pe_two = [0, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115, 0.115]

# Price for the catfishes 
p_one = [0, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665]
p_two = [0, 0.70665, 0.7852, 0.7852, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665, 0.70665]

#survival rate 
sr = [0.8490,1.0]
#sr = [0.8490, 0.9052]

t= len(min_production) 

print("min_production:", len(min_production))
print("h:", len(h))
print("var_cost:", len(var_cost))
print("pe_one:", len(pe_one))
print("pe_two:", len(pe_two))
print("p_one:", len(p_one))
print("p_two:", len(p_two))
print("t:", t)

min_production: 13
h: 13
var_cost: 13
pe_one: 13
pe_two: 13
p_one: 13
p_two: 13
t: 13


In [5]:
###Without uncertainty to the survival rate for the feed 

In [6]:
#########Model Set-up(in english) ###############
# Max (∑p_onet * mt  + ∑ P_twot * max(dt - mt),0) – (12*ft) – (∑ vt*qt) – (∑pe_onet*e_onet + ∑ pe_twot*e_twot) – (∑ ht*xt)
# --->
# Max revenue from the minimum production + revenue from the excess production – total fixed holding cost 
#     –  total variable cost (except feed variable cost) – (total feed variable cost) – total holding cost

# s.t

# q[0] = 0 ---> starting quantity produced at t=0 
# x[0] = 60,000 ---> Starting inventory constraint needs to equal to the minimum d1 
# d[0] = 0 ---> quantity sold at the t=0 
# e_one[0] =0 ---> quantity of feed 1 at the start, t=0
# e_two[0] =0 ---> quantity of feed 2 at the start, t=0
# x[12] >= 60000 ---> inventory at the end, t=12

# xt  = qt + xt-1 - dt ---> Inventory constraint 
# dt  >= mt ---> Catfish sold in month t must be more or equal to the minimum committed quantity of catfish in month t 
# qt <= max_production ---> Production constraint 

#need to minus the holding 
# (∑ pe_onet * e_onet  + ∑ pe_twot * e_twot)   <= max_production_budget – (12 * ft) – (∑ vt*qt) – (∑ ht*xt) 
#  ---> Budget constraint 
# e_onet  + e_twot >= FCR * qt ---> Feed constraint 
# 0.8490 e_onet    +  0.97052 e_twot  – 0.8 (e_onet  + e_twot) >= 0 ---> Survival constraint 

# qt >=0
# xt >=0
# dt >=0
# e_onet  >=0
# e_twot >=0

In [7]:
#########Model Set-up###############

m = Model("production")

### Decision Variables: 
# q = quantity of catfish produced (in pound) in each month t 
q_one = m.addVars(t, name = "quantity_produced_feedone")
q_two = m.addVars(t, name = "quantity_produced_feedtwo")
# x = quantity of catfish (in pound) that will be kept as inventory in each month t
x = m.addVars(t, name = "inventory_kept")
# dt = quantity of catfish (in pound) that will be sold to the intermediaries
d = m.addVars(t, name = "quantity_sold")

# e_one = quantity of feed 1 used in each month t (pound/month) (except month 0 for the planning month) 
e_one = m.addVars(t, name = "quantity_feed_one")
# e_two = quantity of feed 2 used in each month t (pound/month) (except month 0 for the planning month) 
e_two = m.addVars(t, name = "quantity_feed_two")


Academic license - for non-commercial use only - expires 2021-12-19
Using license file /Users/salimwid/gurobi.lic


In [8]:
#to set the objective function 
m.setObjective( ( quicksum(p_one[i]*min_production[i] for i in range(t)) 
                 + quicksum(np.max((d[i]- min_production[i]),0) * p_two[i] for i in range(t)) 
                 -12*f 
                 - quicksum(var_cost[i]*(q_one[i] + q_two[i]) for i in range(t)) 
                - quicksum(pe_one[i]*e_one[i] for i in range(t))
                - quicksum(pe_two[i]*e_two[i] for i in range(t)) - quicksum(h[i]*x[i] for i in range(t)) ), GRB.MAXIMIZE)

In [9]:
#Add the constraints for start, t=0 and t=12
m.addConstr(q_one[0] == 0, "quantity produced at the start, t=0")
m.addConstr(q_two[0] == 0, "quantity produced at the start, t=0")
m.addConstr(x[0] == 60000, "inventory at the start, t=0")
m.addConstr(d[0] == 0 , "quantity sold at the start, t=0")
m.addConstr(e_one[0] == 0, "quantity of feed 1 at the start, t=0")
m.addConstr(e_two[0] == 0, "quantity of feed 2 at the start, t=0")

m.addConstr(x[12] >= 60000, "inventory at the end, t=12")

###Add the inventory constraint 
m.addConstr( ( (x[1] == sr[0]*q_one[1] + sr[1]*q_two[1] + x[0] - d[1]) ) , "inventory1")
m.addConstr( ( (x[2] == sr[0]*q_one[2] + sr[1]*q_two[2] + x[1] - d[2]) ) , "inventory2")
m.addConstr( ( (x[3] == sr[0]*q_one[3] + sr[1]*q_two[3] + x[2] - d[3]) ) , "inventory3")
m.addConstr( ( (x[4] == sr[0]*q_one[4] + sr[1]*q_two[4] + x[3] - d[4]) ) , "inventory4")
m.addConstr( ( (x[5] == sr[0]*q_one[5] + sr[1]*q_two[5] + x[4] - d[5]) ) , "inventory5")
m.addConstr( ( (x[6] == sr[0]*q_one[6] + sr[1]*q_two[6] + x[5] - d[6]) ) , "inventory6")
m.addConstr( ( (x[7] == sr[0]*q_one[7] + sr[1]*q_two[7] + x[6] - d[7]) ) , "inventory7")
m.addConstr( ( (x[8] == sr[0]*q_one[8] + sr[1]*q_two[8] + x[7] - d[8]) ) , "inventory8")
m.addConstr( ( (x[9] == sr[0]*q_one[9] + sr[1]*q_two[9] + x[8] - d[9]) ) , "inventory9")
m.addConstr( ( (x[10] == sr[0]*q_one[10] + sr[1]*q_two[10] + x[9] - d[10]) ) , "inventory10")
m.addConstr( ( (x[11] == sr[0]*q_one[11] + sr[1]*q_two[11] + x[10] - d[11]) ) , "inventory11")
m.addConstr( ( (x[12] == sr[0]*q_one[12] + sr[1]*q_two[12] + x[11] - d[12]) ) , "inventory12")

# Add selling quantity constraint (dt >= mt)
m.addConstrs( ( d[i]  >= min_production[i] for i in range(t) ) ,"Selling_quantity")

#Add production capacity production 
m.addConstrs( ( q_one[i] + q_two[i] <= max_production for i in range(t) ) ,"production_capacity")

#Add budget for feed constraint ((∑ pe_onet * e_onet  + ∑ pe_twot * e_twot) 
# <= max_production_budget – (12 * ft) – (∑ vt*qt)) – (∑ ht*xt)
m.addConstr( ( quicksum(pe_one[i]*e_one[i] + pe_two[i]*e_two[i] for i in range(t)) 
              <= max_production_budget - 12*f -quicksum(var_cost[i]*(q_one[i] + q_two[i]) for i in range(t)) 
                  - quicksum(x[i]*h[i] for i in range(t)) ), "feed_budget")

#Add feed constraint (e_onet  + e_twot >= FCR * qt)
m.addConstrs( ( e_one[i]/ 2.2 >= q_one[i] for i in range(t) ), "feed_constraint1")
m.addConstrs( ( e_two[i]/ 2.2 >= q_two[i] for i in range(t) ), "feed_constraint2")
#m.addConstrs( ( (e_one[i] + e_two[i]) >= 2.2*q[i] for i in range(t) ) ,"feed_constraint")

#Add survival constraint (0.8490 e_onet +  0.97052 e_twot – 0.8(e_onet  + e_twot) >= 0)
m.addConstrs( ( sr[0]*e_one[i] + sr[1]*e_two[i]  >= 0.8*(e_one[i] + e_two[i]) for i in range(t) ), 'Survival')

#Add actual quantity that is produced after considering the survival rate 


{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>}

In [10]:
# Solving the model
m.optimize()

#  Print optimal solutions and optimal value
print("\n Optimal Solution :\n")
for i, v in enumerate(m.getVars()):
    print(v.VarName, v.x)

print('Obj:', m.objVal)

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 85 rows, 78 columns and 245 nonzeros
Model fingerprint: 0xf454cbba
Coefficient statistics:
  Matrix range     [4e-02, 1e+00]
  Objective range  [4e-02, 8e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [6e+04, 7e+06]
Presolve removed 60 rows and 31 columns
Presolve time: 0.04s
Presolved: 25 rows, 47 columns, 117 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9095200e+31   1.200000e+31   6.909520e+01      0s
      27    1.3170683e+05   0.000000e+00   0.000000e+00      0s

Solved in 27 iterations and 0.08 seconds
Optimal objective  1.317068256e+05

 Optimal Solution :

quantity_produced_feedone[0] 0.0
quantity_produced_feedone[1] 0.0
quantity_produced_feedone[2] 0.0
quantity_produced_feedone[3] 0.0
quantity_produced_feedone[4] 0.0
quantity_produced_feedone[5] 0.0
quantity_produced_feedon

In [11]:
solution = np.array(m.x)
len(solution)
solution = solution.reshape(6,13)
df = pd.DataFrame({'q1':solution[0],
                  'q2':solution[1],
                  'inventory':solution[2],
                  'sold':solution[3],
                  'f1':solution[4],
                  'f2':solution[5]})
df

,q1,q2,inventory,sold,f1,f2
0,0.0,0.0,60000.0,0.0,0.0,0.0
1,0.0,96000.0,96000.0,60000.0,0.0,211200.0
2,0.0,96000.0,0.0,192000.0,0.0,211200.0
3,0.0,96000.0,0.0,96000.0,0.0,211200.0
4,0.0,96000.0,0.0,96000.0,0.0,211200.0
5,0.0,96000.0,0.0,96000.0,0.0,211200.0
6,0.0,96000.0,0.0,96000.0,0.0,211200.0
7,0.0,96000.0,0.0,96000.0,0.0,211200.0
8,0.0,96000.0,0.0,96000.0,0.0,211200.0
9,0.0,96000.0,0.0,96000.0,0.0,211200.0
